In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import poisson

In [2]:
data=pd.read_csv(r'C:\Users\enriq\Apple\matches_pl_final.csv')
data.columns = [c.lower() for c in data.columns]

In [3]:
data['comp'].unique()

array(['Premier League', 'Europa Lg', 'EFL Cup', 'FA Cup',
       'Community Shield', 'Champions Lg', 'Conf Lg', 'Super Cup'],
      dtype=object)

In [4]:
data=data[data['comp']=='Premier League']

In [5]:
#Loss or draw = 0, win=1
data["target"] = (data["result"] == "W").astype("int")
data["venue_code"] = data["venue"].astype("category").cat.codes
data["opp_code"] = data["opponent"].astype("category").cat.codes

In [6]:
data

,unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,pass,int,tkl+int,clr,err,season,team,target,venue_code,opp_code
0,0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,...,11.0,9.0,38.0,24.0,0.0,2023,Arsenal,1,0,7
1,1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,...,8.0,8.0,16.0,12.0,0.0,2023,Arsenal,1,1,11
2,2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,...,11.0,10.0,24.0,20.0,1.0,2023,Arsenal,1,0,2
3,3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,...,15.0,7.0,22.0,5.0,1.0,2023,Arsenal,1,1,9
4,4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,...,4.0,8.0,18.0,13.0,0.0,2023,Arsenal,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879,38,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Away,L,0,2,...,5.0,14.0,26.0,9.0,0.0,2022,NorwichCity,0,0,1
1880,39,2022-05-08,14:00,Premier League,Matchweek 36,Sun,Home,L,0,4,...,12.0,9.0,21.0,20.0,1.0,2022,NorwichCity,0,1,21
1881,40,2022-05-11,19:45,Premier League,Matchweek 21,Wed,Away,L,0,3,...,7.0,5.0,24.0,37.0,0.0,2022,NorwichCity,0,0,11
1882,41,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Away,D,1,1,...,3.0,9.0,33.0,23.0,0.0,2022,NorwichCity,0,0,22


In [7]:
#nos importa la la semana 27
data[(data['round']=='Matchweek 27') &  (data['season']==2023)]

,unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,pass,int,tkl+int,clr,err,season,team,target,venue_code,opp_code
36,36,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,Arsenal,0,0,9
89,40,2023-03-11,17:30,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,ManchesterCity,0,0,7
146,43,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,ManchesterUnited,0,1,18
199,38,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,TottenhamHotspur,0,1,17
249,38,2023-03-11,12:30,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,Liverpool,0,0,2
296,33,2023-03-12,16:30,Premier League,Matchweek 27,Sun,Home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,NewcastleUnited,0,1,22
340,31,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,Fulham,0,1,0
383,30,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,BrightonandHoveAlbion,0,0,10
425,27,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,Brentford,0,0,8
474,35,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023,Chelsea,0,0,11


In [8]:
columns=['date', 'time', 'comp', 'round', 'day', 'venue', 'result',
       'gf', 'ga', 'opponent', 'xg_x', 'xga', 'poss','sh_x','sot%','g/sh', 'g/sot','dist','cmp','att_x','cmp%','totdist','prgdist','ast', 'xag', 'xa', 'kp', '1/3', 'ppa', 'crspa','prgp',
        'tklw','tkl%','int','clr','lost','season', 'team', 'target', 'venue_code', 'opp_code']

In [9]:
data=data[columns]

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520 entries, 0 to 1883
Data columns (total 41 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1520 non-null   object 
 1   time        1520 non-null   object 
 2   comp        1520 non-null   object 
 3   round       1520 non-null   object 
 4   day         1520 non-null   object 
 5   venue       1520 non-null   object 
 6   result      1262 non-null   object 
 7   gf          1262 non-null   object 
 8   ga          1262 non-null   object 
 9   opponent    1520 non-null   object 
 10  xg_x        1262 non-null   float64
 11  xga         1262 non-null   float64
 12  poss        1262 non-null   float64
 13  sh_x        1262 non-null   float64
 14  sot%        1262 non-null   float64
 15  g/sh        1262 non-null   float64
 16  g/sot       1220 non-null   float64
 17  dist        1262 non-null   float64
 18  cmp         1262 non-null   float64
 19  att_x       1262 non-null  

In [11]:
dataf=data[data['date']<'2023-03-13']

In [12]:
dataf["g/sot"] = dataf["g/sot"].fillna(dataf["g/sot"].mean())

C:\Users\enriq\AppData\Local\Temp\ipykernel_9004\1122965318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataf["g/sot"] = dataf["g/sot"].fillna(dataf["g/sot"].mean())


In [13]:
def rolling(df,cols,new_cols):
    df=df.sort_values('date')
    rolling_stats=df[cols].rolling(3,closed='left').mean()
    df[new_cols] = rolling_stats
    df = df.dropna(subset=new_cols)
    return df
    

In [14]:
cols=[ "gf", "ga","sot%","g/sot","xg_x","xga","poss",'prgp','g/sh','dist','cmp','att_x','int','clr','lost']

new_cols = [f"{c}_rolling" for c in cols]


camibar dataf por data

In [15]:
matches_rolling = dataf.groupby("team").apply(lambda x: rolling(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')

#aca dropeo

In [16]:
train = matches_rolling[matches_rolling["date"] < '2023-01-01']
test = matches_rolling[(matches_rolling["date"] > '2023-01-01')& (matches_rolling["date"]<'2023-03-11')].dropna()

In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 23 to 600
Data columns (total 56 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           168 non-null    object 
 1   time           168 non-null    object 
 2   comp           168 non-null    object 
 3   round          168 non-null    object 
 4   day            168 non-null    object 
 5   venue          168 non-null    object 
 6   result         168 non-null    object 
 7   gf             168 non-null    object 
 8   ga             168 non-null    object 
 9   opponent       168 non-null    object 
 10  xg_x           168 non-null    float64
 11  xga            168 non-null    float64
 12  poss           168 non-null    float64
 13  sh_x           168 non-null    float64
 14  sot%           168 non-null    float64
 15  g/sh           168 non-null    float64
 16  g/sot          168 non-null    float64
 17  dist           168 non-null    float64
 18  cmp      

In [18]:
model = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [19]:
predictors=["venue_code", "opp_code","sot%","xg_x","xga","poss",'g/sh','dist','cmp','att_x','prgp','int','clr','lost']

In [20]:
predictors,rolling

(['venue_code',
  'opp_code',
  'sot%',
  'xg_x',
  'xga',
  'poss',
  'g/sh',
  'dist',
  'cmp',
  'att_x',
  'prgp',
  'int',
  'clr',
  'lost'],
 <function __main__.rolling(df, cols, new_cols)>)

In [21]:
model.fit(train[predictors],train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [22]:
preds=model.predict(test[predictors])

In [23]:
accuracy = accuracy_score(test["target"], preds)
accuracy

0.7916666666666666

In [24]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-11-01']
    test = data[(data["date"] > '2022-11-01')&(data["date"] < '2023-03-11')].dropna()
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [25]:
combined, precision_score=make_predictions(matches_rolling,predictors)

In [27]:
combined

,actual,predicted
18,1,0
20,1,1
21,1,1
22,1,1
23,0,0
...,...,...
596,1,0
597,0,0
598,0,1
599,0,0
